In [1]:
import os
import glob
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision.models import vgg19
import torch.nn.functional as F


In [10]:
# 1. Dataset Loader ------------------------------------------------------
class MRIDataset(Dataset):
    def __init__(self, root_dir, transform_hr, transform_lr):
        self.hr_images = []
        for class_dir in os.listdir(root_dir):
            self.hr_images += glob.glob(os.path.join(root_dir, class_dir, '*.png'))
        self.transform_hr = transform_hr
        self.transform_lr = transform_lr

    def __len__(self):
        return len(self.hr_images)

    def __getitem__(self, idx):
        img = Image.open(self.hr_images[idx]).convert('RGB')
        hr = self.transform_hr(img)
        lr = self.transform_lr(img)
        return lr, hr

In [11]:
# 2. Generator (MetaSR-style) --------------------------------------------
class MetaSRGenerator(nn.Module):
    def __init__(self, scale):
        super(MetaSRGenerator, self).__init__()
        self.scale = scale
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.ReLU(inplace=True)
        )
        self.res_blocks = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(64, 64, 3, 1, 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 64, 3, 1, 1)) for _ in range(5)]
        )
        self.upsample = nn.Sequential(
            nn.Conv2d(64, 256, 3, 1, 1),
            nn.PixelShuffle(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, 3, 1, 1)
        )

    def forward(self, x):
        x = self.conv1(x)
        res = self.res_blocks(x)
        x = x + res  # skip connection
        return self.upsample(x)

In [12]:
# 3. Discriminator --------------------------------------------------------
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 3, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [13]:
# 4. Weight Prediction Network -------------------------------------------
class WeightPredictor(nn.Module):
    def __init__(self):
        super(WeightPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(16, 1, 3, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [14]:
# 5. Perceptual Loss -----------------------------------------------------
class VGGPerceptualLoss(nn.Module):
    def __init__(self):
        super(VGGPerceptualLoss, self).__init__()
        vgg = vgg19(pretrained=True).features[:36].eval()
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg

    def forward(self, sr, hr):
        return F.l1_loss(self.vgg(sr), self.vgg(hr))

In [15]:
# 6. Training Loop --------------------------------------------------------
def train_metasr_gan(train_loader, num_epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    G = MetaSRGenerator(scale=2).to(device)
    D = Discriminator().to(device)
    W = WeightPredictor().to(device)

    perceptual_loss = VGGPerceptualLoss().to(device)
    bce_loss = nn.BCELoss()
    l1_loss = nn.L1Loss()

    opt_G = torch.optim.Adam(G.parameters(), lr=1e-4)
    opt_D = torch.optim.Adam(D.parameters(), lr=1e-4)

    for epoch in range(num_epochs):
        for i, (lr, hr) in enumerate(train_loader):
            lr, hr = lr.to(device), hr.to(device)

            # Train Discriminator
            D.zero_grad()
            real_out = D(hr)
            fake = G(lr)
            fake_out = D(fake.detach())
            real_loss = bce_loss(real_out, torch.ones_like(real_out))
            fake_loss = bce_loss(fake_out, torch.zeros_like(fake_out))
            d_loss = (real_loss + fake_loss) * 0.5
            d_loss.backward()
            opt_D.step()

            # Train Generator
            G.zero_grad()
            fake_out = D(fake)
            adv_loss = bce_loss(fake_out, torch.ones_like(fake_out))
            content_loss = perceptual_loss(fake, hr)
            pixel_loss = l1_loss(fake, hr)
            g_loss = content_loss + 0.001 * adv_loss + 0.01 * pixel_loss
            g_loss.backward()
            opt_G.step()

            if i % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")



In [ ]:
# 7. Prepare Dataloader ---------------------------------------------------
transform_hr = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transform_lr = transforms.Compose([
    transforms.Resize((112, 112)),  # 2x downsampling
    transforms.Resize((224, 224)),  # simulate upscaling
    transforms.ToTensor()
])

train_dataset = MRIDataset("/kaggle/input/dataset-new/dataset_new/1/Training", transform_hr, transform_lr)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# 8. Launch Training ------------------------------------------------------
train_metasr_gan(train_loader)

# Done. You can extend with validation loop + checkpoint saving next.

In [ ]:
# Diffusion part not needed-----------------------------------------------------

In [36]:
import os
import torch
from torchvision.utils import save_image
from tqdm import tqdm


def denoise_images(model, noise_scheduler, test_loader, device, save_dir, class_names):
    model.eval()
    os.makedirs(save_dir, exist_ok=True)
    with torch.no_grad():
        for i, (img, label) in enumerate(tqdm(test_loader)):
            img = img.to(device)
            noise = torch.randn_like(img)
            t = torch.randint(0, noise_scheduler.config.num_train_timesteps, (1,), device=device).long()
            noisy_img = noise_scheduler.add_noise(img, noise, t)
            denoised = model(noisy_img, t).sample

            for j in range(img.size(0)):  # batch-wise saving
                class_name = class_names[label[j].item()]
                class_dir = os.path.join(save_dir, class_name)
                os.makedirs(class_dir, exist_ok=True)
                save_image(denoised[j], os.path.join(class_dir, f"img_{i}_{j}.png"))


In [37]:
from PIL import Image
import glob


def convert_images_to_rgb_resize(input_dir, output_dir, size=(128, 128)):
    os.makedirs(output_dir, exist_ok=True)
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for file_name in os.listdir(class_path):
            if file_name.endswith(".png"):
                img_path = os.path.join(class_path, file_name)
                img = Image.open(img_path).convert("RGB").resize(size)
                out_class_dir = os.path.join(output_dir, class_name)
                os.makedirs(out_class_dir, exist_ok=True)
                img.save(os.path.join(out_class_dir, file_name))


In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import accuracy_score

def evaluate_keras_model(model_path, img_dir, img_size):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        img_dir,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
    model = load_model(model_path)
    preds = model.predict(generator)
    pred_classes = np.argmax(preds, axis=1)
    true_labels = generator.classes  # ✅ Correct true labels
    acc = accuracy_score(true_labels, pred_classes)
    return acc


In [34]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 128  # Assuming diffusion model and CNN both use 128x128

# Step 1: Define transform (grayscale if single channel)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.Grayscale(num_output_channels=1),  # ensure 1-channel input
    transforms.ToTensor(),
])

# Step 2: Create dataset
test_dataset = datasets.ImageFolder(root='/kaggle/input/dataset-new/dataset_new/1/Testing', transform=transform)

# Step 3: Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [41]:
from tensorflow.keras.models import load_model

keras_model_path = "/kaggle/input/cnn/keras/default/1/brain_tumor_cnn.h5"
model = load_model(keras_model_path)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,910 (91.20 MB)

 Trainable params: 23,907,908 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [42]:
# Paths and config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
denoised_dir = "/kaggle/working/denoised_test"
converted_dir = "/kaggle/working/converted_for_keras"
keras_model_path = "/kaggle/input/cnn/keras/default/1/brain_tumor_cnn.h5"  # Replace with your model
denoise_model = ""

# Run all steps
class_names = test_loader.dataset.classes

denoise_images(denoise_model, noise_scheduler, test_loader, device, denoised_dir, class_names)
convert_images_to_rgb_resize(denoised_dir, converted_dir, (IMG_SIZE, IMG_SIZE))

accuracy = evaluate_keras_model(keras_model_path, converted_dir, IMG_SIZE)
print("Accuracy:", accuracy)


100%|██████████| 41/41 [00:32<00:00,  1.24it/s]


Found 1311 images belonging to 4 classes.


I0000 00:00:1746465522.905759     110 service.cc:148] XLA service 0x7e5cf00094f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746465522.908637     110 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746465522.908660     110 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746465523.058605     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


 4/41 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step

I0000 00:00:1746465524.815395     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step
Accuracy: 0.30892448512585813
